In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Operator
import numpy as np



In [ ]:


nq=3
w = QuantumCircuit(nq)
# w.mcp(np.pi,[0,1,2],3)
diag = np.ones(2**nq)
diag[-1]=-1

w.diagonal(diag.tolist(), list(range(nq)))
w = w.decompose().decompose().decompose().decompose().decompose()
# # w.draw()
# # w.num_nonlocal_gates()
# # w.num_nonlocal_gates()
# 
# # out = transpile(w, basis_gates=['cx', 'rx','ry'])
# print(w.draw())
print(w.reverse_bits().draw())
print(w.global_phase)



# multicontrol Z gate

In [2]:
def get_pattern(n_ctrl, pattern=[0]):
    """
    find control sequence to implement an n_control gate 
    """
    if n_ctrl == pattern[-1]+1:
        return pattern
    else:
        new_pattern = pattern*2
        new_pattern[-1]+=1
        return get_pattern(n_ctrl, new_pattern)

def primitive_u1_block(angle, n_ctrl):
    """
    get primative gates to perform multicontrol rotation
    """
    primitive=QuantumCircuit(2)
    primitive.p(angle, 1)
    primitive.cx(0,1)
    
    return primitive

def multicontrol_Z_gate(ctrl_list, target):
    """
    return circuit that performs an n-control Z gate
    
    """
    # angle set to pi due to Z gate!
    angle = np.pi/ (2**(len(ctrl_list)))
    ctrl_list=sorted(ctrl_list)
    
    full_circ=QuantumCircuit(max([target, max(ctrl_list)])+1)    
    i=0
    while len(ctrl_list)>0:
        circ=QuantumCircuit(max([target, max(ctrl_list)])+1) 
        n_ctrl= len(ctrl_list)
        pattern = get_pattern(n_ctrl)
    
        if n_ctrl >1: 
            for s,i in enumerate(pattern):
                j = ctrl_list[-i-1]
                p = primitive_u1_block((-1)**s*angle, n_ctrl)
                circ = circ.compose(p, [j,target])

            circ = circ.compose(circ)
        else:
            #plus
            circ.p(+angle, target)
            circ.cx(ctrl_list[-1],target)
            # minus
            circ.p(-angle, target)
            circ.cx(ctrl_list[-1],target)
        
        if i ==0 and n_ctrl>1:
            # need correction for first term!
            circ.p(angle, ctrl_list[-1])
            i+=1
        
        full_circ=full_circ.compose(circ)
        # make control list one shorter and set target to last control!
        target = ctrl_list[-1]
        ctrl_list = ctrl_list[:-1]
        
        
    full_circ.p(angle, target)
    return full_circ


In [3]:
ctrl_list = [0,1,5,2] 
target=ctrl_list[-1]+1


circ = multicontrol_Z_gate(ctrl_list,
                           target)


check = QuantumCircuit(max([target, max(ctrl_list)])+1)
check.h(target)
check.mcx(ctrl_list,target)
check.h(target)



np.allclose(Operator(check).data,
            Operator(circ).data)


True

Block encoding ideas:
- use diagonal gate to implement multicontrol Z gate (then use cascade of CNOTs on system reg to apply dense Pauli op)
- use multiplexor rotation gates to implement iX / iY / iZ (then correct phase somewhere // at end of circuit)
